<a href="https://colab.research.google.com/github/GN-Yu/TSRL-project/blob/main/one_dimension.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import psutil
import tensorflow as tf
import subprocess

# Function to execute shell command and return output
def run_shell_command(cmd):
    return subprocess.run(cmd, stdout=subprocess.PIPE, shell=True).stdout.decode('utf-8').strip()

# Check CPU
cpu_info = run_shell_command("cat /proc/cpuinfo | grep 'model name' | uniq")
print("CPU:", cpu_info.split(":")[1].strip() if cpu_info else "Not available")

# Number of CPU Cores
num_cores = os.cpu_count()
print("Number of CPU cores:", num_cores)

# Total RAM
ram_info = psutil.virtual_memory()
total_ram = ram_info.total / (1024 ** 3)  # Convert bytes to GB
print(f"Total RAM : {total_ram} GB")

# Check GPU
gpu_info = !nvidia-smi --query-gpu=gpu_name,memory.total --format=csv
if gpu_info and len(gpu_info) > 1:
    gpu_name = gpu_info[1].split(',')[0]
    gpu_memory_mib = float(gpu_info[1].split()[2])  # Extract the memory in MiB
    gpu_memory_gb = gpu_memory_mib / 1024
    print(f"GPU: {gpu_name}, {gpu_memory_gb} GB")
else:
    print("GPU: Not available")

# TensorFlow info
print(f"TensorFlow version: {tf.__version__}")
print("TensorFlow GPU Availability:", tf.test.is_gpu_available())

CPU: Intel(R) Xeon(R) CPU @ 2.20GHz
Number of CPU cores: 2
Total RAM : 12.678337097167969 GB


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


GPU: Tesla T4, 15.0 GB
TensorFlow version: 2.14.0
TensorFlow GPU Availability: True


In [3]:
# organize files
import os
for dirname, _, filenames in os.walk('/content/drive/MyDrive/TSRL_project'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [4]:
# Set random seed
SEED = 2023
tf.random.set_seed(SEED)

In [5]:
import numpy as np

# Define Constants
X0 = 1  # Initial value for PIDE

lambda_poisson = 0.3
mu = 0.4
sigma = 0.25

T = 1.0  # Total time
N = 50  # Number of time intervals
M = 1000  # Number of samples
Iterations = 400  # Number of iterations

initial_learning_rate = 5e-5

# Generate equally spaced time snapshots from 0 to T
time_steps = np.linspace(0, T, N+1)


In [9]:
def sample_brownian_motion(num_samples, num_intervals, total_time):
    # Implement sampling of Brownian motion here
    pass

def sample_jumps(num_samples, intensity, distribution):
    # Implement jump sampling here
    pass

def sample_initial_states(num_samples):
    # Implement initial state sampling here
    pass



In [10]:
def create_neural_network(input_dim, layer_width=25, output_dim=2):
    # Define the input layer
    inputs = tf.keras.Input(shape=(input_dim,))

    # First linear layer
    x = tf.keras.layers.Dense(layer_width, activation='tanh')(inputs)

    # Add five residual blocks
    for _ in range(5):
        y = tf.keras.layers.Dense(layer_width, activation='tanh')(x)
        y = tf.keras.layers.Dense(layer_width, activation='tanh')(y)
        x = tf.keras.layers.Add()([x, y])

    # Final linear layer
    outputs = tf.keras.layers.Dense(output_dim)(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

In [11]:
d = 1
input_dim = d + 1  # Set 'd' as problem's dimensionality
model = create_neural_network(input_dim)